In [ ]:
from ipywidgets import IntProgress
from IPython.display import display
import time

import os
import sys
module_path = os.path.abspath(os.path.join('./covid_vulnerabilidad_mex'))
if module_path not in sys.path:
    sys.path.append(module_path)

from covid19_vulnerabilidad_mex.datos import *
from covid19_vulnerabilidad_mex.vulnerabilidad import *

import pandas as pd
import geopandas as gpd

import glob
import os
import matplotlib.pyplot as plt
import numpy as np

from datetime import timedelta
import datetime
from datetime import timedelta, date, datetime

from sklearn.feature_selection import SelectKBest, chi2
from sklearn.ensemble import RandomForestRegressor, RandomForestClassifier
from sklearn.feature_selection import SelectFromModel
from sklearn.cross_decomposition import PLSRegression

import imageio
from pygifsicle import optimize
from pathlib import Path


In [ ]:
%load_ext autoreload
%autoreload 2

## Nuevos datos de comorbilidades

Del trabajo de INEGI de estimación en áreas pequeñas

In [ ]:
comorbilidades = pd.read_excel("datos/municipios/a_peq_prev_2018.xlsx",
                               engine='openpyxl',
                               skiprows=2,
                               nrows=12450,
                               usecols='A:I',
                               dtype={'Identificador único del municipio':str,
                                      'Clave de entidad federativa':str,
                                      'Clave de municipio o delegación':str}
                            ).rename({'Identificador único del municipio': 'cvegeo',
                                      'Porcentaje de población de 20 años y más con obesidad.':'pct_obesidad', 
                                      'Porcentaje de población de 20 años y más con diagnóstico previo de hipertensión.':'pct_hipertension',
                                      'Porcentaje de población de 20 años y más con diagnóstico previo de diabetes.':'pct_diabetes'},
                                      axis=1)
comorbilidades = comorbilidades[comorbilidades['Municipio o delegación'] != 'Total']
comorbilidades = comorbilidades[comorbilidades['Estimador'] == 'Valor']
comorbilidades = comorbilidades[['cvegeo', 'pct_obesidad', 'pct_hipertension', 'pct_diabetes']]
comorbilidades


,cvegeo,pct_obesidad,pct_hipertension,pct_diabetes
10,01001,31.486541,14.942242,7.495861
15,01002,32.282284,15.320425,7.953634
20,01003,40.004293,13.751906,9.172624
25,01004,32.596450,16.431493,7.383116
30,01005,34.731715,12.356755,6.745819
...,...,...,...,...
12425,32054,31.075955,16.920279,9.895491
12430,32055,30.526273,21.746589,12.050064
12435,32056,36.822130,20.010453,11.814172
12440,32057,32.477646,15.941678,8.089966


In [ ]:
comorbilidades.columns

Index(['cvegeo', 'pct_obesidad', 'pct_hipertension', 'pct_diabetes'], dtype='object')

In [ ]:
variables_municipales = pd.read_csv('datos/municipios/indicadores_back.csv', index_col=False,
                                    dtype={'cvegeo': str, 'entidad_cv': str,
                                            'municipio_cvegeo': str, 'entidad_cvegeo': str},
                                    encoding='iso8859_2')
variables_municipales.head()

,cvegeo,nom_ent,pt_2015,an_2015,ne614_015,bi_2015,sins_15,vpt_2015,vns_2015,vna_2015,...,hospitales_sme,hospitales_semar,hospitales_sedena,hospitales_pemex,total_hospitales_publicos,total_camas_publicos,hospitales_privados,total_de_camas_privados,total_camas,pob2020
0,1001,Aguascalientes,877190,2.0570,3.4786,25.6873,14.1560,0.5478,0.5543,0.6627,...,0,0,0,0,8,1228,17,43,1271,961977
1,1002,Aguascalientes,46464,4.4207,2.5959,41.8921,5.5355,1.6779,5.6457,1.2979,...,0,0,0,0,0,0,0,0,0,50864
2,1003,Aguascalientes,56048,4.7805,4.3906,49.1856,9.7524,1.0577,0.7756,0.9519,...,0,0,0,0,1,46,1,0,46,60760
3,1004,Aguascalientes,15577,4.3233,2.6265,33.0597,5.1165,1.5629,2.8986,0.5968,...,0,0,0,0,0,0,0,0,0,16918
4,1005,Aguascalientes,120405,3.2445,4.1297,33.7271,13.8341,0.9201,1.0273,0.9443,...,0,0,0,0,1,297,0,0,297,130184


In [ ]:
variables_municipales.columns

Index(['cvegeo', 'nom_ent', 'pt_2015', 'an_2015', 'ne614_015', 'bi_2015',
       'sins_15', 'vpt_2015', 'vns_2015', 'vna_2015', 'vnd_2015', 'vne_2015',
       'vnl_2015', 'vnr_2015', 'irs_2015', 'grs_2015', 'lmex_2015',
       'poblacion', 'pobreza', 'pobreza_pob', 'pobreza_e', 'pobreza_e_pob',
       'pobreza_m', 'pobreza_m_pob', 'vul_car', 'vul_car_pob', 'vul_ing',
       'vul_ing_pob', 'npnv', 'npnv_pob', 'ic_rezedu', 'ic_rezedu_pob',
       'ic_asalud', 'ic_asalud_pob', 'ic_segsoc', 'ic_segsoc_pob', 'ic_cv',
       'ic_cv_pob', 'ic_sbv', 'ic_sbv_pob', 'ic_ali', 'ic_ali_pob',
       'carencias', 'carencias_pob', 'carencias3', 'carencias3_pob', 'plb',
       'plb_pob', 'plbm', 'plbm_pob', 'pob_total', 'mayores_65', 'pob_menore',
       'sin_derech', 'sin_dere_1', 'entidad_cv', 'consultori', 'consulto_1',
       'farmacias', 'id', 'municipio_cvegeo', 'nom_mun', 'porc_carencia_salud',
       'entidad_cvegeo', 'pob01', 'tasa_diabetes', 'tasa_cardiacas',
       'tasa_cancer', 'tasa_pulmo

In [ ]:
variables_nuevas = variables_municipales.merge(comorbilidades, on='cvegeo')
variables_nuevas.drop(columns=[c for c in variables_nuevas.columns if (c.startswith('grado') or c.startswith('tasa'))], inplace=True)
variables_nuevas

,cvegeo,nom_ent,pt_2015,an_2015,ne614_015,bi_2015,sins_15,vpt_2015,vns_2015,vna_2015,...,hospitales_pemex,total_hospitales_publicos,total_camas_publicos,hospitales_privados,total_de_camas_privados,total_camas,pob2020,pct_obesidad,pct_hipertension,pct_diabetes
0,10001,Durango,32852,3.1222,1.6670,40.9181,13.3934,2.6927,8.2132,1.7125,...,0,1,14,3,15,29,34749,40.942458,25.442995,10.923545
1,10002,Durango,4683,11.3703,12.9754,66.9906,16.7841,18.2134,12.7494,5.8109,...,0,0,0,0,0,0,4816,35.246719,16.660050,10.097658
2,10003,Durango,4390,3.7238,2.0022,50.1693,6.7198,2.2059,15.2573,1.7463,...,0,0,0,0,0,0,4649,37.948475,21.286903,10.024307
3,10004,Durango,35415,3.3873,4.0658,43.7066,14.8496,2.5932,7.8444,4.9595,...,0,1,52,0,0,52,37479,41.969079,22.875282,11.749951
4,10005,Durango,654876,1.5050,2.7330,23.1620,14.8938,1.8150,1.1205,1.0511,...,0,7,933,29,291,1224,695597,34.240282,20.334472,10.100384
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2171,32054,Zacatecas,19155,8.1332,3.6565,42.1699,8.8384,2.5438,8.9568,3.8478,...,0,0,0,0,0,0,20177,31.075955,16.920279,9.895491
2172,32055,Zacatecas,30240,4.7985,3.4198,53.7212,13.5516,0.9201,4.8484,1.7459,...,0,1,22,1,0,22,31804,30.526273,21.746589,12.050064
2173,32056,Zacatecas,146147,1.2432,2.1746,23.0304,14.0968,0.2485,1.2004,1.6826,...,0,3,337,5,0,337,155533,36.822130,20.010453,11.814172
2174,32057,Zacatecas,19413,5.6122,2.5997,42.0488,9.0609,1.1950,6.9448,3.0891,...,0,1,12,0,0,12,20285,32.477646,15.941678,8.089966


In [ ]:
variables_nuevas.columns

Index(['cvegeo', 'nom_ent', 'pt_2015', 'an_2015', 'ne614_015', 'bi_2015',
       'sins_15', 'vpt_2015', 'vns_2015', 'vna_2015', 'vnd_2015', 'vne_2015',
       'vnl_2015', 'vnr_2015', 'irs_2015', 'grs_2015', 'lmex_2015',
       'poblacion', 'pobreza', 'pobreza_pob', 'pobreza_e', 'pobreza_e_pob',
       'pobreza_m', 'pobreza_m_pob', 'vul_car', 'vul_car_pob', 'vul_ing',
       'vul_ing_pob', 'npnv', 'npnv_pob', 'ic_rezedu', 'ic_rezedu_pob',
       'ic_asalud', 'ic_asalud_pob', 'ic_segsoc', 'ic_segsoc_pob', 'ic_cv',
       'ic_cv_pob', 'ic_sbv', 'ic_sbv_pob', 'ic_ali', 'ic_ali_pob',
       'carencias', 'carencias_pob', 'carencias3', 'carencias3_pob', 'plb',
       'plb_pob', 'plbm', 'plbm_pob', 'pob_total', 'mayores_65', 'pob_menore',
       'sin_derech', 'sin_dere_1', 'entidad_cv', 'consultori', 'consulto_1',
       'farmacias', 'id', 'municipio_cvegeo', 'nom_mun', 'porc_carencia_salud',
       'entidad_cvegeo', 'pob01', 'hospitales_imss', 'hospitales_issste',
       'hospitales_ssa', 'ho

## Nuevas variables de hospitales

* Eliminar división por sector (isste, imss, etc)
* Sólo: total_hospitales_publicos y privados y los respectivos números de camas
* Agregados por estado porque no tiene mucho sentido por municipio
* Las camas de hospitales privados creo que tienen mucho error, las voy a eliminar

In [ ]:
variables_nuevas.drop(columns=['hospitales_imss',
                               'hospitales_issste', 'hospitales_ssa', 'hospitales_sme',
                               'hospitales_semar', 'hospitales_sedena', 'hospitales_pemex', 'total_camas'], inplace=True)
variables_nuevas["total_camas_publicos_estatal"] = (variables_nuevas[['nom_ent', 'total_camas_publicos']]
                                                    .groupby('nom_ent')
                                                    .transform('sum'))
variables_nuevas["total_camas_privados_estatal"] = (variables_nuevas[['nom_ent', 'total_de_camas_privados']]
                                                    .groupby('nom_ent')
                                                    .transform('sum'))
variables_nuevas["total_hospitales_publicos_estatal"] = (variables_nuevas[['nom_ent', 'total_hospitales_publicos']]
                                                    .groupby('nom_ent')
                                                    .transform('sum'))
variables_nuevas["total_hospitales_privados_estatal"] = (variables_nuevas[['nom_ent', 'hospitales_privados']]
                                                    .groupby('nom_ent')
                                                    .transform('sum'))                                                 
variables_nuevas.drop(columns=['total_camas_publicos', 'total_de_camas_privados',
                               'total_hospitales_publicos', 'hospitales_privados'], inplace=True)
variables_nuevas.rename({"total_camas_publicos_estatal": "total_camas_publicos",
                         "total_camas_privados_estatal": "total_camas_privados",
                         "total_hospitales_publicos_estatal": "total_hospitales_publicos",
                         "total_hospitales_privados_estatal": "total_hospitales_privados"}, axis=1, inplace=True)
variables_nuevas.drop(columns="total_camas_privados")
variables_nuevas

,cvegeo,nom_ent,pt_2015,an_2015,ne614_015,bi_2015,sins_15,vpt_2015,vns_2015,vna_2015,...,entidad_cvegeo,pob01,pob2020,pct_obesidad,pct_hipertension,pct_diabetes,total_camas_publicos,total_camas_privados,total_hospitales_publicos,total_hospitales_privados
0,10001,Durango,32852,3.1222,1.6670,40.9181,13.3934,2.6927,8.2132,1.7125,...,10,31401,34749,40.942458,25.442995,10.923545,1777,390,38,47
1,10002,Durango,4683,11.3703,12.9754,66.9906,16.7841,18.2134,12.7494,5.8109,...,10,4122,4816,35.246719,16.660050,10.097658,1777,390,38,47
2,10003,Durango,4390,3.7238,2.0022,50.1693,6.7198,2.2059,15.2573,1.7463,...,10,4530,4649,37.948475,21.286903,10.024307,1777,390,38,47
3,10004,Durango,35415,3.3873,4.0658,43.7066,14.8496,2.5932,7.8444,4.9595,...,10,33664,37479,41.969079,22.875282,11.749951,1777,390,38,47
4,10005,Durango,654876,1.5050,2.7330,23.1620,14.8938,1.8150,1.1205,1.0511,...,10,582267,695597,34.240282,20.334472,10.100384,1777,390,38,47
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2171,32054,Zacatecas,19155,8.1332,3.6565,42.1699,8.8384,2.5438,8.9568,3.8478,...,32,18490,20177,31.075955,16.920279,9.895491,1122,0,29,49
2172,32055,Zacatecas,30240,4.7985,3.4198,53.7212,13.5516,0.9201,4.8484,1.7459,...,32,29395,31804,30.526273,21.746589,12.050064,1122,0,29,49
2173,32056,Zacatecas,146147,1.2432,2.1746,23.0304,14.0968,0.2485,1.2004,1.6826,...,32,138176,155533,36.822130,20.010453,11.814172,1122,0,29,49
2174,32057,Zacatecas,19413,5.6122,2.5997,42.0488,9.0609,1.1950,6.9448,3.0891,...,32,16934,20285,32.477646,15.941678,8.089966,1122,0,29,49


In [ ]:
variables_nuevas.columns

Index(['cvegeo', 'nom_ent', 'pt_2015', 'an_2015', 'ne614_015', 'bi_2015',
       'sins_15', 'vpt_2015', 'vns_2015', 'vna_2015', 'vnd_2015', 'vne_2015',
       'vnl_2015', 'vnr_2015', 'irs_2015', 'grs_2015', 'lmex_2015',
       'poblacion', 'pobreza', 'pobreza_pob', 'pobreza_e', 'pobreza_e_pob',
       'pobreza_m', 'pobreza_m_pob', 'vul_car', 'vul_car_pob', 'vul_ing',
       'vul_ing_pob', 'npnv', 'npnv_pob', 'ic_rezedu', 'ic_rezedu_pob',
       'ic_asalud', 'ic_asalud_pob', 'ic_segsoc', 'ic_segsoc_pob', 'ic_cv',
       'ic_cv_pob', 'ic_sbv', 'ic_sbv_pob', 'ic_ali', 'ic_ali_pob',
       'carencias', 'carencias_pob', 'carencias3', 'carencias3_pob', 'plb',
       'plb_pob', 'plbm', 'plbm_pob', 'pob_total', 'mayores_65', 'pob_menore',
       'sin_derech', 'sin_dere_1', 'entidad_cv', 'consultori', 'consulto_1',
       'farmacias', 'id', 'municipio_cvegeo', 'nom_mun', 'porc_carencia_salud',
       'entidad_cvegeo', 'pob01', 'pob2020', 'pct_obesidad',
       'pct_hipertension', 'pct_diabetes'

In [ ]:
variables_nuevas.to_csv("datos/municipios/indicadores.csv", index=False)